In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import finesse
from finesse.analysis.actions import RunLocks, Noxaxis, Xaxis

finesse.configure(plotting=True)

In [19]:
kat = finesse.Model()
kat.parse(
    """
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 2500
    var tuning 0.0033834586466165405/2

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 125
    s s0 L0.p1 bs.p1
    
    bs bs T=0.5 L=0 alpha=45

    ###########################################################################
    ###   X arm
    ###########################################################################
    s LX bs.p3 etmx.p1 L=&Larm
    m etmx T=0 L=0 phi=-&tuning

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s LY bs.p2 etmy.p1 L=&Larm
    m etmy T=0 L=0 phi=&tuning
    
    dof DARM etmx.dofs.z -1 etmy.dofs.z +1
    """
)

kat.parse("""

parallel(
    noxaxis(name='tuning'),
    
    series(
    change(tuning.value=0),
    change(DARM.DC=0.0033834586466165405/2),
    noxaxis(name='DARM_DC'),
    )
)

pd Pin bs.p1.i
pd PRefl bs.p1.o
pd Pout bs.p4.o
""")                                          

outtest = kat.run()

print()
print("##################### The results: #####################")
print(f"The input power is {outtest['tuning']['Pin']} W")

print()
print(f"The reflected power with tunings is {outtest['tuning']['PRefl']} W")
print(f"The reflected power with DARM DC is {outtest['series']['DARM_DC']['PRefl']} W")

print()
print(f"The power at AS with tunings is {outtest['tuning']['Pout']} W")
print(f"The power at AS with DARM is {outtest['series']['DARM_DC']['Pout']} W")


##################### The results: #####################
The input power is 125.0 W

The reflected power with tunings is 4.3589962289491957e-07 W
The reflected power with DARM DC is 4.3589962289491957e-07 W

The power at AS with tunings is 124.99999956410039 W
The power at AS with DARM is 124.99999956410039 W


In [20]:
print(outtest)

- Solution Tree
○ parallel - ParallelSolution
├──● tuning - ArraySolution
╰──○ series
   ╰──● DARM_DC - ArraySolution


In [139]:
10e-12/1064e-9*360

0.0033834586466165405

In [16]:
L = 10e-12
lambdaa = 1064e-9
P0 = 125
P0*(1-(np.cos(2*np.pi*L/lambdaa))**2)

4.3589962195333953e-07